In [ ]:
"""This uses AUtogen and it is simpler than Azure ai service because its connecting to 
github model and this is giving result to the user based on availabilty so its checking if a city is 
available then responding otherwise saying I dont know"""

import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

In [2]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=14, completion_tokens=8) cached=False logprobs=None thought=None


In [7]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
city, status = vacation_destinations("Barcelona")
print(f"How about visiting {city}? It's currently {status} there!")

How about visiting Barcelona? It's currently Available there!


In [4]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

In [5]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

In [8]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Barcelona", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=183, completion_tokens=18), content=[FunctionCall(id='call_bpvAuNUlHjcadZRsuUmmEzLV', arguments='{"city":"Barcelona"}', name='vacation_destinations')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content="('Barcelona', 'Available')", call_id='call_bpvAuNUlHjcadZRsuUmmEzLV')], type='ToolCallExecutionEvent')]
source='assistant' models_usage=RequestUsage(prompt_tokens=174, completion_tokens=308) content="Great choice! Barcelona is a vibrant city known for its unique architecture, rich culture, and beautiful beaches. Here are some recommendations for your trip:\n\n### Must-See Attractions:\n1. **Sagrada Família** - Antoni Gaudí's iconic basilica, still under construction, is a must-visit.\n2. **Park Güell** - Another Gaudí masterpiece, this colorful park is filled with artistic structures and provides stunning views of the city.\